In [5]:
import pandas as pd

import warnings

warnings.filterwarnings('ignore')


## Power spectrum

In [3]:

def bend_pl(f, norm, f_bend, alph_lo, alph_hi, sharpness):
    # Bending power-law with two slopes, modified from Summons et al. 2007 
    # (http://adsabs.harvard.edu/abs/2007MNRAS.378..649S)
    # to include 'sharpness' parameter.  Sharpness = 1 same as simpler Summons et al. model, larger values
    # give a sharper transition from one slope to the other.
    # Typical slopes for AGN would be alph_lo=-1, alph_hi=-2 to -3
    powmod = (norm*(f/f_bend)**alph_lo)/(1.+(f/f_bend)**(sharpness*(alph_lo-alph_hi)))**(1./sharpness)
    return powmod

def dbl_bend_pl(f, norm, f_bend_lo, f_bend_hi, alph_lo, alph_med, alph_hi, sharpness):
    # As bend_pl but now with two bends.  If AGN look like BHXRBs the low-frequency slope would be ~0,
    # with medium and high slopes like that for the single-bend case.  This shape could be mimicked by 
    # multiple Lorentzians
    powmod = (norm*(f/f_bend_lo)**alph_lo)/(1.+(f/f_bend_lo)**((sharpness*(alph_lo-alph_med)))*
        (1.+(f/f_bend_hi)**(sharpness*(alph_med-alph_hi))))**(1./sharpness)
    return powmod


## NS models

In [4]:
from NSmodels import *

INFO[2026-02-05 14:56:21,390]: arviz_base not installed
INFO[2026-02-05 14:56:21,391]: arviz_stats not installed
INFO[2026-02-05 14:56:21,392]: arviz_plots not installed


## sims = different simSEED and sampleSEED, everything else same

#### simdata creation

In [5]:
simDATA = pd.DataFrame()
simDATA[['ID','simSEED','sampleSEED', 'rms', 'noiseSIGMA', 'bendfreq', 'lowalpha', 'highalpha', 'sharpness' ]] = -1, -1, -1, -1, -1, -1, -1, -1, -1
simDATA

,ID,simSEED,sampleSEED,rms,noiseSIGMA,bendfreq,lowalpha,highalpha,sharpness


In [6]:
for i in range(10000):
    np.random.seed() 
    rand1 =  np.random.randint(0, 100000)
    np.random.seed() 
    rand2 =  np.random.randint(0, 100000)
    row = [1000+i, rand1, rand2, 0.15, 0.015, 1/365, -1, -2.5, 10]
    simDATA.loc[len(simDATA)] = row

In [36]:
simDATA

,ID,simSEED,sampleSEED,rms,noiseSIGMA,bendfreq,lowalpha,highalpha,sharpness
0,1000.0,59363.0,23136.0,0.15,0.015,0.00274,-1.0,-2.5,10.0
1,1001.0,1765.0,42701.0,0.15,0.015,0.00274,-1.0,-2.5,10.0
2,1002.0,25578.0,22607.0,0.15,0.015,0.00274,-1.0,-2.5,10.0
3,1003.0,99968.0,3820.0,0.15,0.015,0.00274,-1.0,-2.5,10.0
4,1004.0,56490.0,23992.0,0.15,0.015,0.00274,-1.0,-2.5,10.0
...,...,...,...,...,...,...,...,...,...
9995,10995.0,9648.0,72644.0,0.15,0.015,0.00274,-1.0,-2.5,10.0
9996,10996.0,41689.0,10238.0,0.15,0.015,0.00274,-1.0,-2.5,10.0
9997,10997.0,23031.0,13470.0,0.15,0.015,0.00274,-1.0,-2.5,10.0
9998,10998.0,81384.0,2527.0,0.15,0.015,0.00274,-1.0,-2.5,10.0


In [37]:
simDATA[['NightsperWINDOW', 'NumofWINDOW', 'OBSperiod', 'WINDOWwidth' ,'dataLOSSfrac']] = 20, 8, 6, 60, 0.2


In [39]:
simDATA.to_csv('simDATAcsvs/simDATA_varsimSEED_varsampleSEED.csv',index=False)

## sims : low alpha = -1, high alpha = [-2, -4]

#### simdata creation

In [32]:
simDATA = pd.DataFrame()
simDATA[['ID','simSEED','sampleSEED', 'rms', 'noiseSIGMA', 'bendfreq', 'lowalpha', 'highalpha', 'sharpness' ]] = -1, -1, -1, -1, -1, -1, -1, -1, -1
simDATA

,ID,simSEED,sampleSEED,rms,noiseSIGMA,bendfreq,lowalpha,highalpha,sharpness


In [33]:
highALPHA = np.linspace(-2, -4, 6)
lowALPHA = np.array([0, -1])
np.random.seed() 
rand1 = np.random.randint(0,100000, size=100)
np.random.seed() 
rand2 = np.random.randint(0,100000, size=100)
for i in range(600):
    for j in range(2):
        row = [1000+i*2 + j, rand1[i%100], rand2[i%100], 0.15, 0.015, 1/365, lowALPHA[int(j)], highALPHA[int(i/100)] , 10]
        simDATA.loc[len(simDATA)] = row

In [34]:
simDATA[['NightsperWINDOW', 'NumofWINDOW', 'OBSperiod', 'WINDOWwidth' ,'dataLOSSfrac']] = 20, 8, 6, 60, 0.2


In [35]:
simDATA.to_csv('simDATAcsvs/final/simDATA_highALPHA_2_4_low_alpha_0_1.csv',index=False)

## sims : low alpha = -1, NumofWINDOW = [4, 20] highALPHA = -3

#### simdata creation

In [14]:
simDATA = pd.DataFrame()
simDATA[['ID','simSEED','sampleSEED', 'rms', 'noiseSIGMA', 'bendfreq', 'lowalpha', 'highalpha', 'sharpness', 'NumofWINDOW' ]] = -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

In [15]:
#highALPHA = np.linspace(-2, -4, 10)
NumofWINDOW = np.linspace(4, 20, 9)
np.random.seed() 
rand1 = np.random.randint(0,100000, size=100)
np.random.seed()
rand2 = np.random.randint(0,100000, size=100)
for i in range(900):
    
    row = [1000+i, rand1[i%100], rand2[i%100], 0.15, 0.015, 1/365, -1, -3 , 10, int(NumofWINDOW[int(i/100)])]
    simDATA.loc[len(simDATA)] = row

In [16]:
simDATA[['NightsperWINDOW', 'OBSperiod', 'WINDOWwidth' ,'dataLOSSfrac']] = 20, 6, 60, 0.2


In [17]:
simDATA.to_csv('simDATAcsvs/final/simDATA_highALPHA_3_NumofWINDOW_4_20.csv',index=False)

## sims : low alpha = -1, highALPHA = -3, OBSperiod = 12(months), NumofWINDOW = 6, WINDOWwidth = [2, 12]*30 

#### simdata creation

In [18]:
simDATA = pd.DataFrame()
simDATA[['ID','simSEED','sampleSEED', 'rms', 'noiseSIGMA', 'bendfreq', 'lowalpha', 'highalpha', 'sharpness', 'WINDOWwidth' ]] = -1, -1, -1, -1, -1, -1, -1, -1, -1, -1
simDATA

,ID,simSEED,sampleSEED,rms,noiseSIGMA,bendfreq,lowalpha,highalpha,sharpness,WINDOWwidth


In [19]:
#highALPHA = np.linspace(-2, -4, 10)
WINDOWwidth = np.linspace(2*30, 11*30, 6)
np.random.seed() 
rand1 = np.random.randint(0,100000, size=100)
np.random.seed()
rand2 = np.random.randint(0,100000, size=100)
for i in range(600):
    
    row = [1000+i, rand1[i%100], rand2[i%100], 0.15, 0.015, 1/365, -1, -3 , 10, int(WINDOWwidth[int(i/100)])]
    simDATA.loc[len(simDATA)] = row

In [20]:
simDATA[['NightsperWINDOW', 'OBSperiod', 'NumofWINDOW' ,'dataLOSSfrac']] = 40, 12, 6, 0.2


In [21]:
simDATA.to_csv('simDATAcsvs/final/simDATA_highALPHA_3_WINDOWwidth_2_12.csv',index=False)

## sims : low alpha = -1, high alpha = -3, NumofWINDOWS= 6, A/rms = [0.1, 2], period/OBSlength = [1/8, 1]

#### simdata creation

In [22]:
simDATA = pd.DataFrame()
simDATA[['ID','simSEED','sampleSEED', 'rms', 'noiseSIGMA', 'bendfreq', 'lowalpha', 'highalpha', 'sharpness', 'period', 'A1' ]] = -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1
simDATA

,ID,simSEED,sampleSEED,rms,noiseSIGMA,bendfreq,lowalpha,highalpha,sharpness,period,A1


In [23]:
period = np.linspace(1/8, 0.75, 6)
A1 = np.linspace(0.75, 5, 6)
np.random.seed() 
rand1 = np.random.randint(0,100000, size=100)
np.random.seed()
rand2 = np.random.randint(0,100000, size=100)
for i in range(600):
    for j in range(6):
        row = [1000+ i*6 + j, rand1[i%100], rand2[i%100], 0.15, 0.015, 1/365, -1, -3 , 10, period[int(i/100)]* 10, A1[int(j)]*0.15]
        simDATA.loc[len(simDATA)] = row

In [24]:
simDATA.loc[(simDATA.period == 0.25*6) & (simDATA.A1==0.015)]

,ID,simSEED,sampleSEED,rms,noiseSIGMA,bendfreq,lowalpha,highalpha,sharpness,period,A1


In [25]:
simDATA[['NightsperWINDOW', 'OBSperiod', 'WINDOWwidth' ,'dataLOSSfrac', "NumofWINDOW"]] = 15, 6, 60, 0.2, 20

In [26]:
simDATA.to_csv('simDATAcsvs/final/simDATA_highALPHA_3_NumofWINDOW_20_period_1.25_7.5_A1_0.1125_0.75_bendf_1year.csv',index=False)


## sims : all params


#### simdata creation

In [27]:
simDATA = pd.DataFrame()
simDATA[['ID','simSEED','sampleSEED', 'rms', 'noiseSIGMA', 'bendfreq', 'lowalpha', 'highalpha', 'sharpness', 'period', 'A1' , "NumofWINDOW"]] =  -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1
simDATA

,ID,simSEED,sampleSEED,rms,noiseSIGMA,bendfreq,lowalpha,highalpha,sharpness,period,A1,NumofWINDOW


In [28]:
#highALPHA = np.linspace(-2, -4, 10)
period = np.linspace(1/8, 1, 4)
A1 = np.array([0] + list(np.linspace(0.5, 2.5, 4)))
bend_freq = 1/(np.linspace(2, .25, 4) * 365) # 1/ days
highALPHA = np.linspace(-2, -4, 3)
NumofWINDOW = np.linspace(4, 20, 4 )

In [29]:
simDATA[['NightsperWINDOW', 'OBSperiod', 'WINDOWwidth' ,'dataLOSSfrac', ]] = 20, 6, 60, 0.2

In [31]:
simDATA.loc[(simDATA.highalpha == -3) & (simDATA.NumofWINDOW == NumofWINDOW[2]) & (simDATA.A1 == 0.0)].to_csv('simDATAcsvs/final/simDATA_highALPHA_3_NumofWINDOW_14_A1_0.0_bendfreq_2_0.25.csv',index=False)